In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import (
    EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3,
    EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7,
    EfficientNetV2B0, EfficientNetV2B1, EfficientNetV2B2, EfficientNetV2B3,
    EfficientNetV2S, EfficientNetV2M, EfficientNetV2L
)
from tensorflow.keras.applications.efficientnet import preprocess_input as effnetv1_preprocess
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as effnetv2_preprocess
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from IPython.display import display

# Set seeds for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Constants
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
BATCH_SIZE = 32
EPOCHS = 64
IMG_SIZE = (224, 224)

# Data Generators (no augmentation)
def get_generators(preprocess_func):
    train_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func
    ).flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        seed=SEED
    )

    test_gen = ImageDataGenerator(
        preprocessing_function=preprocess_func
    ).flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    return train_gen, test_gen

# Build classifier
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    return Model(inputs=base_model.input, outputs=output)

# EfficientNet Models
models = {
    'EfficientNetB0': (EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB1': (EfficientNetB1(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB2': (EfficientNetB2(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB3': (EfficientNetB3(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB4': (EfficientNetB4(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB5': (EfficientNetB5(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB6': (EfficientNetB6(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetB7': (EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv1_preprocess),
    'EfficientNetV2B0': (EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2B1': (EfficientNetV2B1(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2B2': (EfficientNetV2B2(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2B3': (EfficientNetV2B3(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2S': (EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2M': (EfficientNetV2M(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
    'EfficientNetV2L': (EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(224,224,3)), effnetv2_preprocess),
}

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2)

# Result storage
report_rows = []

# Training and Evaluation Loop
for model_name, (base_model, preprocess_func) in models.items():
    print(f"\n🔧 Training model: {model_name}")
    train_gen, test_gen = get_generators(preprocess_func)
    model = build_model(base_model)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=test_gen,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )

    predictions = model.predict(test_gen, verbose=0)
    predicted_labels = (predictions > 0.5).astype(int).ravel()
    true_labels = test_gen.classes

    accuracy = accuracy_score(true_labels, predicted_labels)
    report = classification_report(true_labels, predicted_labels, digits=6, output_dict=True)

    row = {
        'Model': model_name,
        'Accuracy': accuracy,
        'Macro Precision': report['macro avg']['precision'],
        'Macro Recall': report['macro avg']['recall'],
        'Macro F1': report['macro avg']['f1-score'],
        'Weighted Precision': report['weighted avg']['precision'],
        'Weighted Recall': report['weighted avg']['recall'],
        'Weighted F1': report['weighted avg']['f1-score'],
    }
    report_rows.append(row)

# Save and display CSV
df_report = pd.DataFrame(report_rows)
csv_path = '/kaggle/working/classification_summary_no_aug.csv'
df_report.to_csv(csv_path, index=False)
print(f"✅ Saved summary to {csv_path}")
print("📊 Classification Summary:")
display(df_report)


2025-07-20 12:18:59.631516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753013939.796639      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753013939.845073      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1753013952.009339      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
31790344/31790344 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
165234480/165234480 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step
24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
28456008/28456008 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
35839040/35839040 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
52606240/52606240 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step
473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step

🔧 Training model: EfficientNetB0
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64


I0000 00:00:1753014083.678267      59 service.cc:148] XLA service 0x7cee64002a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753014083.679160      59 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753014085.628329      59 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753014093.961313      59 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
163/163 - 159s - 978ms/step - accuracy: 0.9172 - loss: 0.1886 - val_accuracy: 0.8381 - val_loss: 0.4111 - learning_rate: 0.0010
Epoch 2/64
163/163 - 50s - 306ms/step - accuracy: 0.9517 - loss: 0.1256 - val_accuracy: 0.7997 - val_loss: 0.6106 - learning_rate: 0.0010
Epoch 3/64
163/163 - 51s - 314ms/step - accuracy: 0.9551 - loss: 0.1158 - val_accuracy: 0.8606 - val_loss: 0.4420 - learning_rate: 0.0010
Epoch 4/64
163/163 - 50s - 307ms/step - accuracy: 0.9632 - loss: 0.0958 - val_accuracy: 0.8221 - val_loss: 0.5712 - learning_rate: 2.0000e-04

🔧 Training model: EfficientNetV2L
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/64
163/163 - 201s - 1s/step - accuracy: 0.9045 - loss: 0.2159 - val_accuracy: 0.8109 - val_loss: 0.4335 - learning_rate: 0.0010
Epoch 2/64
163/163 - 50s - 308ms/step - accuracy: 0.9446 - loss: 0.1373 - val_accuracy: 0.8093 - val_loss: 0.4887 - learning_rate: 0.0010
Epoch 3/64
163/163 - 50s - 306ms/step - accuracy: 0.9515 - loss: 0.1193 - val_accuracy: 0.8141 - val_loss: 0.4855 - learning_rate: 0.0010
Epoch 4/64
163/163 - 51s - 314ms/step - accuracy: 0.9617 - loss: 0.1048 - val_accuracy: 0.8045 - val_loss: 0.5521 - learning_rate: 2.0000e-04
✅ Saved summary to /kaggle/working/classification_summary_no_aug.csv
📊 Classification Summary:


,Model,Accuracy,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1
0,EfficientNetB0,0.862179,0.876727,0.829915,0.844577,0.868131,0.862179,0.857653
1,EfficientNetB1,0.825321,0.864581,0.774786,0.792866,0.846461,0.825321,0.813364
2,EfficientNetB2,0.836538,0.859548,0.794872,0.811353,0.847583,0.836538,0.828585
3,EfficientNetB3,0.841346,0.892806,0.790171,0.810721,0.869604,0.841346,0.829755
4,EfficientNetB4,0.899038,0.915681,0.872222,0.887089,0.905296,0.899038,0.896272
5,EfficientNetB5,0.887821,0.901571,0.860684,0.874689,0.892922,0.887821,0.884830
6,EfficientNetB6,0.884615,0.902392,0.854701,0.870192,0.891726,0.884615,0.881010
7,EfficientNetB7,0.842949,0.876098,0.798291,0.816735,0.859733,0.842949,0.834063
8,EfficientNetV2B0,0.866987,0.888385,0.832051,0.848688,0.876378,0.866987,0.861843
9,EfficientNetV2B1,0.873397,0.900169,0.837179,0.855240,0.885566,0.873397,0.868057
